<a href="https://colab.research.google.com/github/benryanx/skin-cancer-detection-ai/blob/main/Skin_Cancer_Detection_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, I will use the HAM10000 dataset ("Human Against Machine with 10,000 images"),
which is the industry benchmark for skin lesion classification.

# 1. Data Ingestion & Setup

In [ ]:
# --- STEP A: SETUP KAGGLE & DOWNLOAD DATA ---
import os
from google.colab import files

# Upload your kaggle.json here
if not os.path.exists("/root/.kaggle/kaggle.json"):
    files.upload()
    !mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Download the HAM10000 dataset
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
!unzip -q skin-cancer-mnist-ham10000.zip

# --- STEP B: THE PYTORCH PIPELINE ---
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# 1. Preprocessing (Medical Imaging Standard)
# We resize to 224x224 and use the 'ImageNet' mean/std for normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. Transfer Learning: Load ResNet50
# We use the pre-trained 'weights' from ImageNet
model = models.resnet50(weights='DEFAULT')

# Freeze all layers except the final decision layer
for param in model.parameters():
    param.requires_grad = False

# Re-wire for 7 types of skin lesions (Melanoma, Nevi, etc.)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)

# --- STEP C: THE PREDICTION FUNCTION ---
from PIL import Image

def predict_skin_lesion(image_path):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img_t = transform(img).unsqueeze(0) # Prepare for the brain

    with torch.no_grad():
        output = model(img_t)
        prediction = torch.argmax(output).item()

    classes = ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']
    print(f"AI Diagnostic Result: {classes[prediction]}")

print("Pipeline Ready! You can now train on the folders or upload a test image.")

Upload an image for skin check

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt

# 1. TRIGGER THE UPLOAD
print("Please upload a photo of a skin lesion (.jpg or .png):")
uploaded = files.upload()

for filename in uploaded.keys():
    # 2. RUN PREDICTION
    # This calls the function we defined in the previous step
    model.eval()
    img = Image.open(filename).convert('RGB')

    # Preprocess the image for the model
    img_t = transform(img).unsqueeze(0)

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    img_t = img_t.to(device)

    with torch.no_grad():
        output = model(img_t)
        # Apply softmax to get probabilities
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        prediction = torch.argmax(probabilities).item()
        confidence = probabilities[prediction].item() * 100

    # 3. SHOW THE RESULTS
    classes = ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']

    plt.imshow(img)
    plt.title(f"AI Prediction: {classes[prediction]} ({confidence:.2f}% Confidence)")
    plt.axis('off')
    plt.show()

    print(f"Diagnostic Code: {classes[prediction]}")
    print(f"Confidence: {confidence:.2f}%")

We can train the model even more now to get better accuracy

To start we run the train loader:
Context
the "Conveyor Belt" that feeds images to your brain hasn't been built yet.
To fix this, we need to create the train_loader. However, the HAM10000 dataset is famously "messy"—the images are stored in two separate zip folders, and the labels are in a CSV file. You can't just point the model at the images; you have to organize them first so PyTorch understands which image is a mole and which is cancer.

# 2. Model Training

In [ ]:
import os
import shutil
import pandas as pd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. ORGANIZE THE FILES (The HAM10000 "Secret Sauce")
# PyTorch's ImageFolder needs files to be in folders named after their class
metadata = pd.read_csv("HAM10000_metadata.csv")
base_dir = 'organized_skin_data'
os.makedirs(base_dir, exist_ok=True)

# Create sub-folders for each of the 7 classes
classes = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
for cls in classes:
    os.makedirs(os.path.join(base_dir, cls), exist_ok=True)

# Copy images into their respective class folders
# Note: HAM10000 images are usually in two folders: part_1 and part_2
image_folders = ['HAM10000_images_part_1', 'HAM10000_images_part_2']
for folder in image_folders:
    if os.path.exists(folder):
        for img_name in os.listdir(folder):
            img_id = img_name.split('.')[0]
            # Find the label for this specific image ID
            label = metadata.loc[metadata['image_id'] == img_id, 'dx'].values[0]
            shutil.copy(os.path.join(folder, img_name), os.path.join(base_dir, label, img_name))

# 2. DEFINE THE TRAIN_LOADER
# This is what your error was missing!
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the newly organized folder as a PyTorch Dataset
full_dataset = datasets.ImageFolder(root=base_dir, transform=transform)

# Create the iterable DataLoader
# batch_size=32 means the "Conveyor Belt" delivers 32 images at a time
train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)

# Define the 'device' so the model knows to use the GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Train Loader is now defined and images are organized!")

Next is the optimizer:
we haven't yet introduced the "Teacher" (optimizer) or the "Scoring System" (criterion) to this specific part of the code.
**In PyTorch, the Optimizer is the mathematical engine that performs the Calculus for you**. It looks at the errors and decides exactly how much to nudge each weight in the neural network to improve the next guess.

In [ ]:
import torch.optim as optim

# 1. THE SCORING SYSTEM (Loss Function)
# CrossEntropyLoss is the gold standard for classification.
# It measures the "distance" between the AI's probability and the truth.
criterion = nn.CrossEntropyLoss()

# 2. THE TEACHER (Optimizer)
# 'Adam' is a very popular optimizer because it automatically adjusts the
# learning rate as it goes.
# We only pass 'model.fc.parameters()' because we are only training
# the new final layer we added.
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 3. MOVE MODEL TO DEVICE
# This ensures your model is sitting on the GPU so it can handle the math.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Optimizer and Criterion are ready! You can now run the Training Loop.")

Next we run the training:

In [ ]:
# --- THE "BRAIN" TRAINING LOOP ---
num_epochs = 5  # Start with 5 'laps' through the data
model.train()   # Put the model in learning mode

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Move data to the GPU (if available) for 10x speed
        images, labels = images.to(device), labels.to(device)

        # 1. Clear previous errors (Gradients)
        optimizer.zero_grad()

        # 2. Forward Pass: Make a guess
        outputs = model(images)

        # 3. Calculate the "Penalty" (Loss)
        loss = criterion(outputs, labels)

        # 4. BACKPROPAGATION: The Calculus Step!
        # This finds out which weights to blame for the loss
        loss.backward()

        # 5. Step: The Optimizer nudges the weights to be better
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} finished. Error level: {running_loss/len(train_loader):.4f}")

Let's save the "brain" so no need to retrain.
Save the model’s weights to a file in your Google Drive so you can "load" them instantly without re-training

In [ ]:
# Save the 'State Dict' (the actual weights) of your model
torch.save(model.state_dict(), 'skin_cancer_model.pth')
print("Model weights saved! Download this file to keep your 'trained' brain.")

# Congratulations!! You know have an Ai that can identify different types of Skin Cancers! 🧑